<a href="https://colab.research.google.com/github/765302995/PyLake_v2/blob/main/USGS_LAKE_contuine_dissolved_oxygen_tempure_get.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U dataretrieval

In [ ]:
!pip install cubo[ee]

  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.6/242.6 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 2.3 MB/s eta 0:00:00
  Created wheel for cubo: filename=cubo-2024.8.0-py3-none-any.whl size=10159 sha256=2a3e71a22aabff38ed37377e3f375d6bf11c2bf75d577e9c3fcc35f40646b3e1
  Stored in directory: /root/.cache/

In [8]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import dataretrieval.nwis as nwis
import pandas as pd
import time
from datetime import datetime, timedelta

data = {
    'State': [
        'Alabama', 'Alaska', 'American Samoa', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut',
        'Delaware',
        'District of Columbia', 'Federated States of Micronesia', 'Florida', 'Georgia', 'Guam', 'Hawaii', 'Idaho',
        'Illinois',
        'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Marshall Islands', 'Maryland',
        'Massachusetts', 'Michigan',
        'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey',
        'New Mexico',
        'New York', 'North Carolina', 'North Dakota', 'Northern Mariana Is.', 'Ohio', 'Oklahoma', 'Oregon', 'Palau',
        'Pennsylvania',
        'Puerto Rico', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont',
        'Virginia',
        'Virgin Islands', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming'
    ],
    'Ab': [
        'AL', 'AK', 'AS', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FM', 'FL', 'GA', 'GU', 'HI', 'ID', 'IL', 'IN',
        'IA', 'KS',
        'KY', 'LA', 'ME', 'MH', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC',
        'ND', 'MP',
        'OH', 'OK', 'OR', 'PW', 'PA', 'PR', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'VI', 'WA', 'WV', 'WI',
        'WY'
    ]
}

# 创建 DataFrame
df_STATE_CODE = pd.DataFrame(data)
site_Dataframe = pd.DataFrame()
site_type = "LK"

for i in df_STATE_CODE['Ab']:

    try:
        sites2 = nwis.what_sites(siteType=site_type, stateCd=i, outputDataTypeCd="iv",
                                  parameterCd=["00010", "00300", "00301"])
        sites3_30 = sites2[0][sites2[0]['parm_cd'] == '00300']
        # 将结果添加到 site_Dataframe 中
        site_Dataframe = pd.concat([site_Dataframe, sites3_30], ignore_index=True)
        # 打印stite3_30有多少列
        print("{} have {} lakes dissloved oxygen ".format(i, sites3_30.shape[0]))
        # 增加计数器

    except:
        print("{} have 0 lakes dissloved oxygen ".format(i))
# 根据site_no列是否有重复值，把site_Dataframe分成两个dataframe,第一个是除去重复的数据，第二个是重复的数据
unique_site_df = site_Dataframe.drop_duplicates(subset=['site_no'], keep=False)
unique_site_df.to_csv('unique_site_df.csv')
# 创建一个新的 DataFrame 包含重复值
duplicate_site_df = site_Dataframe[site_Dataframe.duplicated(subset=['site_no'], keep=False)]
duplicate_site_df.to_csv('duplicate_site_df.csv')
failed_sites = []

# 处理 unique_site_df
for index, row in unique_site_df.iterrows():
    site_no = row['site_no']
    begin_date = row['begin_date']
    end_date = row['end_date']

    try:
        # 获取数据
        data = nwis.get_iv(site=site_no, parameterCd=["00010", "00300", "00301"], start=begin_date, end=end_date)
        data[0].to_csv('{}.csv'.format(site_no))
        print("Index {}, finished {}".format(index, site_no))
    except Exception as e:
        print("Error processing site {}: {}".format(site_no, e))
        failed_sites.append(site_no)

    # 休眠 5 秒以避免请求过于频繁
    time.sleep(5)
print("well down in single depth lake!!!")
# 处理 duplicate_site_df
for index, row in duplicate_site_df.iterrows():
    site_no = row['site_no']
    begin_date = row['begin_date']
    end_date = row['end_date']

    try:
        # 获取数据
        data = nwis.get_iv(site=site_no, parameterCd=["00010", "00300", "00301"], start=begin_date, end=end_date)
        data[0].to_csv('{}.csv'.format(site_no))
        print("Index {}, finished {}".format(index, site_no))
    except Exception as e:
        print("Error processing site {}: {}".format(site_no, e))
        failed_sites.append(site_no)

    # 休眠 5 秒以避免请求过于频繁
    time.sleep(5)
print("well down in mutiplit depth lake!!!")
# 打印失败的站点
if failed_sites:
    print("Failed sites: {}".format(failed_sites))
else:
    print("All sites processed successfully")

AL have 0 lakes dissloved oxygen 
AK have 0 lakes dissloved oxygen 
AS have 0 lakes dissloved oxygen 
AZ have 35 lakes dissloved oxygen 
AR have 26 lakes dissloved oxygen 
CA have 6 lakes dissloved oxygen 
CO have 3 lakes dissloved oxygen 
CT have 0 lakes dissloved oxygen 
DE have 0 lakes dissloved oxygen 
DC have 0 lakes dissloved oxygen 
FM have 0 lakes dissloved oxygen 
FL have 1 lakes dissloved oxygen 
GA have 1 lakes dissloved oxygen 
GU have 0 lakes dissloved oxygen 
HI have 0 lakes dissloved oxygen 
ID have 1 lakes dissloved oxygen 
IL have 2 lakes dissloved oxygen 
IN have 6 lakes dissloved oxygen 
IA have 0 lakes dissloved oxygen 
KS have 3 lakes dissloved oxygen 
KY have 0 lakes dissloved oxygen 
LA have 3 lakes dissloved oxygen 
ME have 0 lakes dissloved oxygen 
MH have 0 lakes dissloved oxygen 
MD have 1 lakes dissloved oxygen 
MA have 2 lakes dissloved oxygen 
MI have 0 lakes dissloved oxygen 
MN have 0 lakes dissloved oxygen 
MS have 14 lakes dissloved oxygen 
MO have 1 l

In [7]:
!ls

01073389.csv	     352550114390701.csv  360310114172376.csv  401352074002701.csv
01080000.csv	     352550114390702.csv  360310114172381.csv  403859074555201.csv
01104425.csv	     352550114390703.csv  360314114450505.csv  404028073384601.csv
01594418.csv	     352550114390704.csv  360314114450510.csv  405610074382301.csv
01670144.csv	     352550114390705.csv  360314114450515.csv  413704087153501.csv
01670209.csv	     352550114390706.csv  360314114450520.csv  415026070563603.csv
016702576.csv	     352550114390707.csv  360314114450525.csv  415642111172701.csv
0208458892.csv	     352550114390708.csv  360314114450530.csv  415812111203901.csv
0208458893.csv	     352550114390709.csv  360314114450535.csv  415941111175401.csv
02168500.csv	     352550114390710.csv  360314114450540.csv  415955111212401.csv
02171000.csv	     352550114390711.csv  360314114450545.csv  420446111183201.csv
02344671.csv	     352550114390712.csv  360314114450550.csv  421410121492000.csv
03071590.csv	     360246114443001.c

In [19]:
import shutil
import glob
import os

# 创建目标文件夹，如果不存在
destination_folder = './drive/MyDrive/USGS_continue_DO'
os.makedirs(destination_folder, exist_ok=True)

# 查找并移动所有 CSV 文件
for file_path in glob.glob('/content/*.csv'):
    shutil.move(file_path, destination_folder)

print("所有 CSV 文件已成功移动到 '/content/drive/USGS_continue_DO' 文件夹。")



所有 CSV 文件已成功移动到 '/content/drive/USGS_continue_DO' 文件夹。


In [22]:
import glob
import os
print(len(os.listdir('./drive/MyDrive/USGS_continue_DO')))

196


In [23]:
data_muti_example = pd.read_csv('./drive/MyDrive/USGS_continue_DO/02168500.csv')
data_muti_example

,datetime,site_no,00010_top,00010_top_cd,00010_bottom,00010_bottom_cd,00300_top,00300_top_cd,00300_bottom,00300_bottom_cd
0,2007-10-01 05:00:00+00:00,2168500,24.1,A,16.6,A,7.3,A,0.2,A
1,2007-10-01 06:00:00+00:00,2168500,24.0,A,16.6,A,6.5,A,0.2,A
2,2007-10-01 07:00:00+00:00,2168500,24.0,A,16.7,A,6.8,A,0.2,A
3,2007-10-01 08:00:00+00:00,2168500,24.0,A,16.7,A,6.9,A,0.2,A
4,2007-10-01 09:00:00+00:00,2168500,23.9,A,16.7,A,7.1,A,0.2,A
...,...,...,...,...,...,...,...,...,...,...
319722,2024-02-27 16:15:00+00:00,2168500,11.6,P,10.5,P,10.9,P,9.7,P
319723,2024-02-27 16:30:00+00:00,2168500,11.5,P,10.5,P,11.0,P,9.7,P
319724,2024-02-27 16:45:00+00:00,2168500,-999999.0,"P, ***",10.5,P,10.8,P,9.6,P
319725,2024-02-27 17:00:00+00:00,2168500,NaN,NaN,10.6,P,-999999.0,"P, ***",9.8,P


from matplotlib import pyplot as plt
data_muti_example['00010_top'].plot(kind='hist', bins=20, title='00010_top')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
data_muti_example['00010_bottom'].plot(kind='hist', bins=20, title='00010_bottom')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
data_muti_example['00300_top'].plot(kind='hist', bins=20, title='00300_top')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
data_muti_example['00300_bottom'].plot(kind='hist', bins=20, title='00300_bottom')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
data_muti_example.groupby('00300_top_cd').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
data_muti_example.groupby('00300_bottom_cd').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
data_muti_example.plot(kind='scatter', x='00010_top', y='00010_bottom', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
data_muti_example.plot(kind='scatter', x='00010_bottom', y='00300_top', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
data_muti_example.plot(kind='scatter', x='00300_top', y='00300_bottom', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)